# Getting Started with PySCF

This notebook is meant as an introduction to the PySCF package for those with at least some exposure to Python.

Make sure that you have the following installed:

* Python 2 or 3 (this notebook should work with both)
* IPython/Jupyter
* PySCF (and it's dependencies)

Misc imports to add at the beginning of a script:

In [1]:
# Get rid of the pesky warnings
# If you see capture at the top of a cell it's used to suppress the output of that cell to keep the notebook neat.
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Here we import two modules from PySCF the gto (gaussian type orbital) and scf (self consistent field) modules.
from pyscf import gto, scf, molden
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

%matplotlib notebook

OSError: dlopen(/Users/jets/apps/pyscf/pyscf/lib/libcgto.dylib, 6): Library not loaded: libcint.3.0.dylib
  Referenced from: /Users/jets/apps/pyscf/pyscf/lib/libcgto.dylib
  Reason: image not found

# Molecule Input

There are a few different ways that we can input molecules into PySCF.

* First, we can intialize a molecule (`Mole`) object, add properties to it individually, *and then* build the molecule with the `build` function.

In [ ]:
mol = gto.Mole()
mol.atom = '''O 0 0 0; H  0 1 0; H 0 0 1'''
mol.basis = 'sto-3g'
mol.build()

* Second, we can combine the last two steps by passing the properties like `atom` to the `build` command.

In [ ]:
mol = gto.Mole()
mol.build(
    atom = '''O 0 0 0; H  0 1 0; H 0 0 1''',
    basis = 'sto-3g',
)

* Finally, we can combine all three steps by using the `M` function.

In [ ]:
mol = gto.M(
    atom = '''O 0 0 0; H  0 1 0; H 0 0 1''',
    basis = 'sto-3g',
)

Now that we know the basics of building a molecule in PySCF, let's dig a little deeper into the `atom` property (i.e. the coordinates of the atoms in our system).

If you've worked with quantum chemistry packages before, then you are probably also familiar with the frustration that inputting geometries bring. PySCF makes this as simple as cutting and pasting the coordinates from an XYZ file. In the example below, I simply paste the XYZ coordinates for ethane into the triple-quotes and then set my `atom` equal to the `xyz` variable when I build my molecule.

In [ ]:
mol = None # Clear the variable
xyz = '''
  C     -0.7516      0.0225      0.0209
  C      0.7516     -0.0225     -0.0209
  H      1.1669      0.8330     -0.5693
  H      1.1155     -0.9329     -0.5145
  H      1.1851     -0.0039      0.9875
  H     -1.1669     -0.8334      0.5687
  H     -1.1157      0.9326      0.5151
  H     -1.1850      0.0044     -0.9875'''

mol = gto.Mole()
mol.atom = xyz

The `mol` object also allows to input the basis for the calculation. In the earlier examples I chose the `sto-3g` basis, but in practice you will probably use other basis. For a full list of the basis sets see [this link](https://github.com/sunqm/pyscf/blob/master/pyscf/gto/basis/__init__.py).

In [ ]:
mol.basis = 'ccpvdz' # We usually use this basis set for initial calculations on a system
mol.build()

The molecule object have a few other important attributes which can similarly be set by `mol.property = value` or adding `property = value` to the `build` or `M` functions.
For example we can set the charge, spin, and symmetry of the system as well as the verbosity of the output. The properties I've mentioned here are just the ones that we use commonly in research. For more details about the symmetry see [this link](http://sunqm.github.io/pyscf/symm.html?highlight=symmetry) (this will become important if you need to specify the number of electrons in orbitals of a particular symmetry).

In [ ]:
mol.charge =0
mol.spin = 0 # (number of alpha elec) - (number of beta elec) i.e. for doublets spin = 1
mol.symmetry = True # Allows PySCF to use highest possible symmetry for molecule

# First calculation

For the first calculation we will perform a simple restricted Hartree-Fock calculation on our ethane molecule. First we create our Hartree-Fock object, and by convention we call it `mf` (since DFT is based on a modified Fock operator it is also called `mf` by convention).

In [ ]:
mf = scf.RHF(mol)

The `kernel` function is used to actually launch the calculation of an object (here: minimizes the energy by performing iterative unitary transformations on the molecular orbitals).

In [ ]:
mf.kernel() # Should print -79.2346189538153

We can capture the output of the kernel to save the energy to a variable, or access it at a later time with `mf.e_tot`. Once the optimization is complete our `mf` object also stores the MO energies, occupancy, and coefficients (the columns of the `mf.mo_coeff` array are the MO orbitals).

In [ ]:
print("Optimized energy = %f" % mf.e_tot)
print("Shape of MO coefficient matrix = ", mf.mo_coeff.shape)

By saving the orbital coefficients to a `molden` file, we can view the orbitals with viewers like `Jmol`, `GaussView`, `Avogadro`, etc. This is crucial when picking an active space where we might want to pick all atomic orbitals of a certain type (e.g. `C 2pz`).

In [ ]:
with open('C2H6mo.molden', 'w') as f1:
    molden.header(mol, f1)
    # If you don't specify the ene and occ options, the orbitals will be assigned integral energies and won't show
    # any occupation.
    molden.orbital_coeff(mol, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)

# Note: Molecule Drawing in Python

In [ ]:
# Just a sketch to illustrate how you can roughly plot the molecules within python (w/ only matplotlib)
# You should be able to move/rotate the figure with your mouse, if you can't try running this cell again
cxs = [-0.7516, 0.7516]
hxs = [1.1669, 1.1155, 1.1851, -1.1669, -1.1157, -1.1850]
cys = [ 0.0225, -0.0225]
hys = [0.8330, -0.9329, -0.0039, -0.8334,  0.9326,  0.0044]
czs = [ 0.0209, -0.0209]
hzs = [-0.5693,-0.5145, 0.9875, 0.5687, 0.5151,-0.9875]

# Figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Add Atoms
ax.scatter(cxs,cys,czs, c='#6f6f6f', s=300, depthshade=False)
ax.scatter(hxs,hys,hzs, c='#eaeaea', s=150, depthshade=False)

# Change background color
ax.set_facecolor('#8bd9ff')

plt.show()